
# Clasificación con Random Forest Classifier — **Producción Alta vs Baja**

En este cuaderno convertimos el problema agrícola en **clasificación binaria**:  
- **1 = Producción Alta**  
- **0 = Producción Baja**  

Se usa un **Random Forest Classifier** con preprocesamiento de variables numéricas y categóricas.



## Paso 0 · Requisitos
- Archivo: `1. datos_agricolas.xlsx`, hoja `Inicio`
- Librerías: `pandas`, `scikit-learn`, `matplotlib`
- Target binario creado a partir de la **mediana de Producción_Ton**


In [ ]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



## Paso 1 · Cargar datos y crear variable objetivo binaria
Se define `Produccion_Alta` en base a la mediana de `Producción_Ton`.


In [ ]:

file_path = "1. datos_agricolas.xlsx"  # Ajusta ruta si es necesario
df = pd.read_excel(file_path, sheet_name="Inicio")

# Crear variable objetivo binaria
threshold = df["Producción_Ton"].median()
df["Produccion_Alta"] = (df["Producción_Ton"] > threshold).astype(int)

df[["Producción_Ton", "Produccion_Alta"]].head()



## Paso 2 · Selección de variables predictoras y división Train/Test
- Target: `Produccion_Alta`  
- Features: `Año`, `Mes`, `Región`, `Cultivo`, `Superficie_Ha`, `Precio_Ton`, `Costos_Insumos`


In [ ]:

target_col = "Produccion_Alta"
feature_cols = ["Año", "Mes", "Región", "Cultivo", "Superficie_Ha", "Precio_Ton", "Costos_Insumos"]

X = df[feature_cols]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

len(X_train), len(X_test), y.value_counts()



## Paso 3 · Preprocesamiento
Se aplican transformaciones según el tipo de variable:
- Numéricas: imputación por mediana + escalado
- Categóricas: imputación por moda + one-hot encoding


In [ ]:

numeric_features = ["Año", "Mes", "Superficie_Ha", "Precio_Ton", "Costos_Insumos"]
categorical_features = ["Región", "Cultivo"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)



## Paso 4 · Entrenamiento del **Random Forest Classifier**
Se usa un bosque de 300 árboles.


In [ ]:

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)



## Paso 5 · Evaluación del modelo
- Métricas: Accuracy, reporte de clasificación, matriz de confusión.


In [ ]:

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}\n")

print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Baja","Alta"], yticklabels=["Baja","Alta"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()



## Paso 6 · Importancia de variables


In [ ]:

# Recuperar nombres de features tras el one-hot
ohe = clf.named_steps["preprocessor"].named_transformers_["cat"].named_steps["onehot"]
cat_names = list(ohe.get_feature_names_out(categorical_features))
num_names = numeric_features
all_features = num_names + cat_names

rf = clf.named_steps["model"]
importances = rf.feature_importances_

imp_df = pd.DataFrame({"feature": all_features, "importance": importances})\
          .sort_values("importance", ascending=False)

imp_df.head(15)


In [ ]:

plt.figure(figsize=(8,6))
plt.barh(imp_df["feature"].head(15)[::-1], imp_df["importance"].head(15)[::-1])
plt.xlabel("Importancia")
plt.title("Importancias de características (Top 15)")
plt.tight_layout()
plt.show()



## Paso 7 · Conclusiones
- El problema se transformó en clasificación binaria (Alta vs Baja Producción).  
- Random Forest entrega métricas de exactitud y muestra qué variables son más relevantes para predecir.  
- Próximos pasos: ajustar hiperparámetros, probar clasificación multiclase (por cultivo o región), o balancear clases si son desiguales.
